In [8]:
import gzip
import os
import datetime
import math
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook

In [2]:
sess = None

def reset_tf(sess = None, log_device_placement = False):
    if sess:
        sess.close()
    tf.reset_default_graph()
    tf.set_random_seed(0)
    return tf.InteractiveSession(config = tf.ConfigProto(log_device_placement = log_device_placement))

def dump_statistics():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        print('parameters for "%s": %d' % (variable.name, variable_parameters))
        total_parameters += variable_parameters
    print('total parameters: %d' % total_parameters)

In [105]:
class HyperParameters:
    learning_rate = 1e-3
    
    dropout_rate = 0.0
    
    context_size = 850
    question_size = 60
    answers_size = 6
    
    d_hidden = 200
    
    num_attn_layers_contexts = 2
    num_attn_layers_questions = 2
    num_attn_layers_joint = 2

    dataset_batch_size = 64
    dataset_num_parallel_calls = 4
    dataset_prefetch_size = 128
    dataset_shuffle_size = 1000
    
    max_distance_bias = 15
    
    gradient_clip_norm = 5

In [129]:
class AttentionModel:
    def __init__(self, session, word_embeddings, hparams):
        self._session = session
        self._word_embeddings = word_embeddings
        self._hparams = hparams
        
    def _parse_example(self, example_proto):
        # parse proto
        parsed = tf.parse_single_example(example_proto, features = {
            'context': tf.VarLenFeature(tf.int64),
            'question': tf.VarLenFeature(tf.int64),
            'answer_starts': tf.VarLenFeature(tf.int64),
            'answer_ends': tf.VarLenFeature(tf.int64), })
        
        # convert to dense tensors
        context = tf.sparse_tensor_to_dense(parsed['context'])
        question = tf.sparse_tensor_to_dense(parsed['question'])
        answer_starts = tf.sparse_tensor_to_dense(parsed['answer_starts'])
        answer_ends = tf.sparse_tensor_to_dense(parsed['answer_ends'])
        
        # pad tensors
        context_len = tf.shape(context)[0]
        question_len = tf.shape(question)[0]
        answers_len = tf.shape(answer_starts)[0]
        zero_vector = self._word_embeddings.shape[0] - 1
        context = tf.pad(
            context,
            [[0, self._hparams.context_size - context_len]],
            constant_values = 0)
        question = tf.pad(
            question,
            [[0, self._hparams.question_size - question_len]],
            constant_values = 0)
        answer_starts = tf.pad(
            answer_starts,
            [[0, self._hparams.answers_size - answers_len]],
            constant_values = -1)
        answer_ends = tf.pad(
            answer_ends,
            [[0, self._hparams.answers_size - answers_len]],
            constant_values = -1)
        
        return (context, context_len, question, question_len, answer_starts, answer_ends)
    
    def _build_dataset_pipeline(self):
        with tf.variable_scope('dataset'):
            # placeholders
            self._dataset_filenames = tf.placeholder(
                tf.string,
                shape = [None],
                name = 'dataset_filenames')
            self._dataset_limit = tf.placeholder_with_default(
                tf.constant(-1, tf.int64),
                shape = [],
                name = 'dataset_limit')
            self._dataset_shuffle_size = tf.placeholder_with_default(
                tf.constant(self._hparams.dataset_batch_size, tf.int64),
                shape = [],
                name = 'dataset_shuffle_size')
            self._dataset_batch_size = tf.placeholder_with_default(
                tf.constant(self._hparams.dataset_batch_size, tf.int64),
                shape = [],
                name = 'dataset_batch_size')
            self._dataset_prefetch_size = tf.placeholder_with_default(
                tf.constant(self._hparams.dataset_prefetch_size, tf.int64),
                shape = [],
                name = 'dataset_prefetch_size')

            # build dataset
            dataset = tf.data.TFRecordDataset(
                tf.random_shuffle(self._dataset_filenames),
                compression_type='GZIP')
            dataset = dataset.take(self._dataset_limit)
            dataset = dataset.map(
                self._parse_example,
                num_parallel_calls = self._hparams.dataset_num_parallel_calls)
            dataset = dataset.shuffle(self._dataset_shuffle_size)
            dataset = dataset.prefetch(self._dataset_prefetch_size)
            dataset = dataset.batch(self._dataset_batch_size)

            # build iterator
            self._dataset_iterator = dataset.make_initializable_iterator()
            (contexts,
             context_lens,
             questions,
             question_lens,
             answer_starts,
             answer_ends) = self._dataset_iterator.get_next()
            
            # trim tensors for efficiency
            c_size = tf.reduce_max(context_lens)
            q_size = tf.reduce_max(question_lens)
            contexts = contexts[:, :c_size]
            questions = questions[:, :q_size]
            
            # give key tensors names
            self._contexts = tf.identity(contexts, 'contexts')
            self._context_lens = tf.identity(context_lens, 'context_lens')
            self._questions = tf.identity(questions, 'questions')
            self._question_lens = tf.identity(question_lens, 'question_lens')
            self._answer_starts = tf.identity(answer_starts, 'answer_starts')
            self._answer_ends = tf.identity(answer_ends, 'answer_ends')

            # masks
            self._context_masks = tf.sequence_mask(
                self._context_lens,
                maxlen = c_size,
                dtype = tf.float32,
                name = 'context_masks')
            self._question_masks = tf.sequence_mask(
                self._question_lens,
                maxlen = q_size,
                name = 'question_masks',
                dtype = tf.float32)

            # minibatch size
            self._minibatch_size = tf.shape(self._contexts)[0]
            self._minibatch_size = tf.identity(self._minibatch_size, 'minibatch_size')
    
    def _self_attention(self, layer):
        # sizes
        a = layer
        a_len = tf.shape(a)[1]
        a_size = a.shape[-1].value

        # project
        a = tf.layers.dense(a, a_size, use_bias = False, name = 'proj')

        # compute weights
        a_T = tf.transpose(a, perm = [0, 2, 1])         # [batch, a_size, a_len]
        w = tf.matmul(a, a_T)                           # [batch, a_len, a_len]
        w /= np.sqrt(a_size)

        # mask diagonal
        w += tf.diag(tf.fill([a_len], -1e25))

        # softmax
        w = tf.nn.softmax(w, name = 'weights')

        # compute result
        return tf.matmul(w, layer)
    
    def _cross_attention(self, c, q):
        # sizes
        c_len = tf.shape(c)[1]
        q_len = tf.shape(q)[1]
        size = c.shape[-1].value
        assert q.shape[-1].value == size
        
        # project
        cp = tf.layers.dense(c, size, use_bias = False, name = 'c_proj')
        qp = tf.layers.dense(q, size, use_bias = False, name = 'q_proj')
        qp_T = tf.transpose(qp, perm = [0, 2, 1])       # [batch, size, q_len]
        
        # compute c2q weights
        w_c2q = tf.matmul(cp, qp_T)                     # [batch, c_len, q_len]
        w_c2q /= np.sqrt(size)
        w_q2c = tf.transpose(w_c2q, perm = [0, 2, 1])   # [batch, q_len, c_len]
        
        # softmax
        w_c2q = tf.nn.softmax(w_c2q)
        w_q2c = tf.nn.softmax(w_q2c)
        
        c2q_attn = tf.matmul(w_c2q, q)                  # [batch, c_len, size]
        q2c_attn = tf.matmul(w_q2c, c)                  # [batch, q_len, size]
        
        return c2q_attn, q2c_attn
    
    def _norm(self, layer, name = 'norm'):
        return tf.layers.batch_normalization(layer, training = self._training, name = name)
    
    # pulled from transformer
    def _get_timing_signal_1d(self,
                              length,
                              channels,
                              min_timescale=1.0,
                              max_timescale=1.0e4):
        position = tf.to_float(tf.range(length))
        num_timescales = channels // 2
        log_timescale_increment = (
          math.log(float(max_timescale) / float(min_timescale)) /
          (tf.to_float(num_timescales) - 1))
        inv_timescales = min_timescale * tf.exp(
          tf.to_float(tf.range(num_timescales)) * -log_timescale_increment)
        scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(inv_timescales, 0)
        signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
        signal = tf.pad(signal, [[0, 0], [0, tf.mod(channels, 2)]])
        signal = tf.reshape(signal, [1, length, channels])
        return signal

    def _build_model(self):
        with tf.variable_scope('model'):
            # placeholders
            self._training = tf.placeholder(tf.bool, name = 'training')
            
            # embed contexts
            with tf.variable_scope('embed'):
                word_embeddings = tf.get_variable(
                    name = "word_embeddings",
                    shape = self._word_embeddings.shape,
                    initializer = tf.constant_initializer(self._word_embeddings),
                    trainable = False)
                c = tf.nn.embedding_lookup(
                    word_embeddings,
                    self._contexts)
                c = tf.layers.dense(
                    c,
                    self._hparams.d_hidden)
            
            # embed questions
            with tf.variable_scope('embed', reuse = True):
                q = tf.nn.embedding_lookup(
                    word_embeddings,
                    self._questions)
                q = tf.layers.dense(
                    q,
                    self._hparams.d_hidden)
            
            c_pos = self._get_timing_signal_1d(
                tf.shape(self._contexts)[1],
                self._hparams.d_hidden)
            q_pos = self._get_timing_signal_1d(
                tf.shape(self._questions)[1],
                self._hparams.d_hidden)
            
            c_mask = tf.expand_dims(self._context_masks, axis = -1)
            q_mask = tf.expand_dims(self._question_masks, axis = -1)
            
            c += c_pos
            q += q_pos
            c *= c_mask
            q *= q_mask
            c = self._norm(c, name = 'norm_c')
            q = self._norm(q, name = 'norm_q')
            
            for i in range(2):
                with tf.variable_scope('layer_%d' % i):
                    with tf.variable_scope('self'):
                        c += self._self_attention(c)
                        c = self._norm(c)
                        c *= c_mask
                    with tf.variable_scope('self', reuse = True):
                        q += self._self_attention(q)
                        q = self._norm(q)
                        q *= q_mask
                        
            with tf.variable_scope('cross'):
                c2q, q2c = self._cross_attention(c, q)
                c2q *= c_mask
                q2c *= q_mask
                
            with tf.variable_scope('summarize'):
                w = tf.layers.dense(q2c, 1, use_bias = False)
                w = tf.squeeze(w, axis = -1)
                w = tf.nn.softmax(w)
                w = tf.expand_dims(w, axis = -1)
                q_sum = q2c * w
                q_sum = tf.reduce_sum(q_sum, axis = -2)
                q_sum = tf.expand_dims(q_sum, 1)
                q_sum = tf.tile(q_sum, [1, tf.shape(c)[1], 1])

            m = tf.concat([c, c2q, q_sum], axis = -1)
            
            with tf.variable_scope('memory'):
                m += self._self_attention(m)
                m = self._norm(m)
                m *= c_mask
            
#             # context self-attention layers
#             for i in range(self._hparams.num_attn_layers_contexts):
#                 with tf.variable_scope('encode_%d' % i):
#                     c = self._encoding_layer(c, c_pos)
#
#             # question self-attention layers
#             for i in range(self._hparams.num_attn_layers_questions):
#                 with tf.variable_scope('encode_%d' % i, reuse = True):
#                     q = self._encoding_layer(q, q_pos)
#
#             # joint attention layer
#             with tf.variable_scope('joint_attn'):
#                 attn = self._attention_layer(
#                     queries = c,
#                     keys = q,
#                     values = q)
#                 joint_layer = c + attn
#                 joint_layer = tf.layers.batch_normalization(
#                     joint_layer,
#                     training = self._training)
#               
#             # joint self-attention layers
#             for i in range(self._hparams.num_attn_layers_joint):
#                 with tf.variable_scope('joint_attn_self_%d' % i):
#                     attn = self._attention_layer(
#                         joint_layer,
#                         joint_layer,
#                         joint_layer,
#                         distance_bias = True,
#                         mask_type = 's')
#                     joint_layer += attn
#                     joint_layer = tf.layers.batch_normalization(
#                         joint_layer,
#                         training = self._training)

            # output: start/end logits
            self._answer_start_logits = tf.layers.dense(
                m,
                1,
                use_bias = False,
                name = 'answer_start_logits')
            self._answer_start_logits = tf.squeeze(    # [batch_size, context_size]
                self._answer_start_logits,
                axis = -1,
                name = 'answer_start_logits') 
#             self._answer_end_logits = tf.layers.dense(
#                 joint_layer,
#                 1,
#                 use_bias = False,
#                 name = 'answer_end_logits')
#             self._answer_end_logits = tf.squeeze(      # [batch_size, context_size]
#                 self._answer_end_logits,
#                 axis = -1,
#                 name = 'answer_end_logits')

    def _build_optimizer(self):
        with tf.variable_scope('optimize'):
            # individual losses
            # TODO: incorporate other answers into training
            l0 = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = self._answer_starts[:, 0],
                logits = self._answer_start_logits)
#             l1 = tf.nn.sparse_softmax_cross_entropy_with_logits(
#                 labels = self._answer_ends[:, 0],
#                 logits = self._answer_end_logits)

            # total loss
            self._total_loss = tf.reduce_sum(l0) # + tf.reduce_sum(l1)
            self._total_loss = tf.identity(self._total_loss, 'total_loss')
            
            # mean loss
            self._mean_loss = self._total_loss / tf.cast(self._minibatch_size, tf.float32)
            self._mean_loss = tf.identity(self._mean_loss, 'mean_loss')
            
            # start/end probabilities/estimates
            self._answer_start_probs = tf.nn.softmax(
                self._answer_start_logits,
                name = 'answer_start_logits')
            self._answer_start_estimates = tf.argmax(
                self._answer_start_probs,
                axis = -1,
                name = 'answer_start_estimates')
#             self._answer_end_probs = tf.nn.softmax(
#                 self._answer_end_logits,
#                 name = 'answer_end_logits')
#             # N.B., mask impossible answers
#             mask = 1.0 - tf.sequence_mask(
#                 self._answer_start_estimates,
#                 self._hparams.context_size,
#                 dtype = tf.float32)
#             self._answer_end_estimates = tf.argmax(
#                 mask * self._answer_end_probs,
#                 axis = -1,
#                 name = 'answer_end_estimates')
            
            # exact match accuracy
#             answer_starts_eq = tf.equal(
#                 self._answer_starts[:, 0],
#                 self._answer_start_estimates)
#             answer_ends_eq = tf.equal(
#                 self._answer_ends[:, 0],
#                 self._answer_end_estimates)
#             answers_eq = tf.logical_and(
#                 answer_starts_eq,
#                 answer_ends_eq)
            answers_eq = tf.equal(
                self._answer_starts[:, 0],
                self._answer_start_estimates)
            self._total_exact_matches = tf.reduce_sum(
                tf.cast(answers_eq, tf.int64),
                name = 'total_exact_matches')
            
#             # F1
#             a0 = self._answer_starts[:, 0]
#             a1 = self._answer_ends[:, 0] + 1
#             answer_lens = a1 - a0
#             b0 = self._answer_start_estimates
#             b1 = self._answer_end_estimates + 1
#             answer_estimate_lens = b1 - b0
#             tps = tf.maximum(
#                 tf.cast(0, tf.int64),
#                 tf.minimum(a1, b1) - tf.maximum(a0, b0))
#             fps = answer_estimate_lens - tps
#             fns = answer_lens - tps
#             self._total_true_positives = tf.reduce_sum(
#                 tps,
#                 name = 'total_true_positives')
#             self._total_false_positives = tf.reduce_sum(
#                 fps,
#                 name = 'total_false_positives')
#             self._total_false_negatives = tf.reduce_sum(
#                 fns,
#                 name = 'total_false_negatives')
            
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                self._global_step = tf.Variable(0, name = 'global_step', trainable = False)
                self._optimizer = tf.train.AdamOptimizer(learning_rate = self._hparams.learning_rate)
                
                # gradient clipping
                gradients, variables = zip(*self._optimizer.compute_gradients(self._mean_loss))
                gradients, _ = tf.clip_by_global_norm(
                    gradients, 
                    self._hparams.gradient_clip_norm)
                
                self._train_op = self._optimizer.apply_gradients(
                    zip(gradients, variables),
                    global_step = self._global_step)
                
    def process(self,
                dataset_filenames,
                dataset_limit = -1,
                header = 'results',
                train = False,
                log_file = None):
        # initialize dataset to files
        self._session.run(self._dataset_iterator.initializer, feed_dict={
            self._dataset_filenames: dataset_filenames,
            self._dataset_limit: dataset_limit })

        cum_loss = 0
        cum_num_examples = 0
        cum_exact_matches = 0
        
        # start progress
        start = datetime.datetime.now()
        progress = tqdm_notebook(leave = False, desc = header)

        while True:
            # process a minibatch
            try:
                (_,
                 curr_total_loss,
                 curr_exact_matches,
                 curr_minibatch_size) = self._session.run(
                    (self._train_op if train else (),
                     self._total_loss,
                     self._total_exact_matches,
                     self._minibatch_size),
                    feed_dict = { self._training: train })
            except tf.errors.OutOfRangeError:
                break

            # update loss stats
            cum_loss += curr_total_loss
            cum_exact_matches += curr_exact_matches
            cum_num_examples += curr_minibatch_size
            
            # update progress
            progress.update(curr_minibatch_size)
            progress.set_postfix(loss = cum_loss / cum_num_examples)

        # end progress
        progress.close()
        finish = datetime.datetime.now()
        
        # print/log output
        message = '%s: time=%s, step=%d, loss=%g, exact_match=%g' % (
            header,
            finish - start,
            tf.train.global_step(sess, self._global_step),
            cum_loss / cum_num_examples,
            cum_exact_matches / cum_num_examples)
        print(message)
        if log_file:
            print(message, file=log_file)
            log_file.flush()

In [5]:
with gzip.open('../../data/SQuAD/data_1.vocab.embeddings.npy.gz', 'rb') as f:
    word_embeddings = np.load(f)

In [ ]:
def list_files(path):
    return sorted([os.path.join(path, file) for file in os.listdir(path)])

train_set = list_files('../../data/SQuAD/data_1.train')
dev_set = list_files('../../data/SQuAD/data_1.dev')

In [130]:
sess = reset_tf(sess)

model = AttentionModel(sess, word_embeddings, HyperParameters())
model._build_dataset_pipeline()
model._build_model()
model._build_optimizer()
dump_statistics()

parameters for "model/embed/dense/kernel:0": 60000
parameters for "model/embed/dense/bias:0": 200
parameters for "model/norm_c/gamma:0": 200
parameters for "model/norm_c/beta:0": 200
parameters for "model/norm_q/gamma:0": 200
parameters for "model/norm_q/beta:0": 200
parameters for "model/layer_0/self/proj/kernel:0": 40000
parameters for "model/layer_0/self/norm/gamma:0": 200
parameters for "model/layer_0/self/norm/beta:0": 200
parameters for "model/layer_1/self/proj/kernel:0": 40000
parameters for "model/layer_1/self/norm/gamma:0": 200
parameters for "model/layer_1/self/norm/beta:0": 200
parameters for "model/cross/c_proj/kernel:0": 40000
parameters for "model/cross/q_proj/kernel:0": 40000
parameters for "model/summarize/dense/kernel:0": 200
parameters for "model/memory/proj/kernel:0": 360000
parameters for "model/memory/norm/gamma:0": 600
parameters for "model/memory/norm/beta:0": 600
parameters for "model/answer_start_logits/kernel:0": 600
total parameters: 583800


In [133]:
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(100):
    model.process(train_set[:1], header = str(i), dataset_limit = 500, train = True)

HBox(children=(IntProgress(value=1, bar_style='info', description='0', max=1), HTML(value='')))

0: time=0:00:00.790605, step=8, loss=6.13289, exact_match=0.006


HBox(children=(IntProgress(value=1, bar_style='info', description='1', max=1), HTML(value='')))

1: time=0:00:00.807460, step=16, loss=5.16803, exact_match=0.042


HBox(children=(IntProgress(value=1, bar_style='info', description='2', max=1), HTML(value='')))

...

HBox(children=(IntProgress(value=1, bar_style='info', description='94', max=1), HTML(value='')))

94: time=0:00:00.775796, step=760, loss=0.808775, exact_match=0.728


HBox(children=(IntProgress(value=1, bar_style='info', description='95', max=1), HTML(value='')))

95: time=0:00:00.804191, step=768, loss=0.849484, exact_match=0.722


HBox(children=(IntProgress(value=1, bar_style='info', description='96', max=1), HTML(value='')))

In [9]:
with open('../../logs/SQuAD/model_attention_1.1.log', 'wt') as f:
    for i in range(5):
        model.process(
            train_set,
            header = 'train_%d' % i,
            train = True,
            log_file = f)
        model.process(
            dev_set,
            header = 'dev_%d' % i,
            train = False,
            log_file = f)

HBox(children=(IntProgress(value=1, bar_style='info', description='train_0', max=1), HTML(value='')))

train_0: time=0:07:05.388448, step=1369, loss=8.55389, exact_match=0.0172262, precision=0.0300183, recall=0.212827, F1=0.0526154


HBox(children=(IntProgress(value=1, bar_style='info', description='dev_0', max=1), HTML(value='')))

dev_0: time=0:00:17.065388, step=1369, loss=8.34677, exact_match=0.0216651, precision=0.0274626, recall=0.223968, F1=0.0489259


HBox(children=(IntProgress(value=1, bar_style='info', description='train_1', max=1), HTML(value='')))

train_1: time=0:07:04.695773, step=2738, loss=8.21938, exact_match=0.024966, precision=0.0304504, recall=0.220174, F1=0.0535015


HBox(children=(IntProgress(value=1, bar_style='info', description='dev_1', max=1), HTML(value='')))

KeyboardInterrupt: 

In [54]:
sess.run(
    model._dataset_iterator.initializer,
    feed_dict = {
        model._dataset_filenames: train_set[:1],
        model._dataset_limit: 10 })

In [55]:
contexts, questions, answer_starts, answer_ends, answer_start_estimates, answer_end_estimates = sess.run(
    [model._contexts,
     model._questions,
     model._answer_starts,
     model._answer_ends,
     model._answer_start_estimates,
     model._answer_end_estimates],
    feed_dict = { model._training: False })

In [56]:
answer_start_estimates

array([ 29,   4,  78,  11,  27,  23, 123, 117,  52,  47])

In [57]:
answer_end_estimates

array([ 29,   5,  78,  43,  34,  25, 124, 117,  55, 110])

In [58]:
answer_starts[:, 0]

array([ 65,   4,  78,  49,  80, 181, 123, 117,  52, 110])

In [59]:
answer_ends[:, 0]

array([ 67,   5,  78,  49,  80, 189, 124, 117,  55, 110])

In [20]:
tf.equal(tf.range(10), tf.range(10)).eval()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [80]:
tf.sequence_mask([5, 6, 7], maxlen = 10, dtype = tf.float32).eval()

array([[1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.]], dtype=float32)